In [1]:
from __future__ import print_function
import numpy as np

class RBM:
  
  def __init__(self, num_visible, num_hidden):
    self.num_hidden = num_hidden
    self.num_visible = num_visible
    self.debug_print = True

    # Initialize a weight matrix, of dimensions (num_visible x num_hidden), using
    # a uniform distribution between -sqrt(6. / (num_hidden + num_visible))
    # and sqrt(6. / (num_hidden + num_visible)). One could vary the 
    # standard deviation by multiplying the interval with appropriate value.
    # Here we initialize the weights with mean 0 and standard deviation 0.1. 
    # Reference: Understanding the difficulty of training deep feedforward 
    # neural networks by Xavier Glorot and Yoshua Bengio
    np_rng = np.random.RandomState(1234)

    self.weights = np.asarray(np_rng.uniform(
			low=-0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                       	high=0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                       	size=(num_visible, num_hidden)))


    # Insert weights for the bias units into the first row and first column.
    self.weights = np.insert(self.weights, 0, 0, axis = 0)
    self.weights = np.insert(self.weights, 0, 0, axis = 1)

  def train(self, data, max_epochs = 1000, learning_rate = 0.1):
    """
    Train the machine.
    Parameters
    ----------
    data: A matrix where each row is a training example consisting of the states of visible units.    
    """

    num_examples = data.shape[0]

    # Insert bias units of 1 into the first column.
    data = np.insert(data, 0, 1, axis = 1)

    for epoch in range(max_epochs):      
      # Clamp to the data and sample from the hidden units. 
      # (This is the "positive CD phase", aka the reality phase.)
      pos_hidden_activations = np.dot(data, self.weights)      
      pos_hidden_probs = self._logistic(pos_hidden_activations)
      pos_hidden_probs[:,0] = 1 # Fix the bias unit.
      pos_hidden_states = pos_hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
      # Note that we're using the activation *probabilities* of the hidden states, not the hidden states       
      # themselves, when computing associations. We could also use the states; see section 3 of Hinton's 
      # "A Practical Guide to Training Restricted Boltzmann Machines" for more.
      pos_associations = np.dot(data.T, pos_hidden_probs)

      # Reconstruct the visible units and sample again from the hidden units.
      # (This is the "negative CD phase", aka the daydreaming phase.)
      neg_visible_activations = np.dot(pos_hidden_states, self.weights.T)
      neg_visible_probs = self._logistic(neg_visible_activations)
      neg_visible_probs[:,0] = 1 # Fix the bias unit.
      neg_hidden_activations = np.dot(neg_visible_probs, self.weights)
      neg_hidden_probs = self._logistic(neg_hidden_activations)
      # Note, again, that we're using the activation *probabilities* when computing associations, not the states 
      # themselves.
      neg_associations = np.dot(neg_visible_probs.T, neg_hidden_probs)

      # Update weights.
      self.weights += learning_rate * ((pos_associations - neg_associations) / num_examples)

      error = np.sum((data - neg_visible_probs) ** 2)
      if self.debug_print:
        print("Epoch %s: error is %s" % (epoch, error))

  def run_visible(self, data):
    """
    Assuming the RBM has been trained (so that weights for the network have been learned),
    run the network on a set of visible units, to get a sample of the hidden units.
    
    Parameters
    ----------
    data: A matrix where each row consists of the states of the visible units.
    
    Returns
    -------
    hidden_states: A matrix where each row consists of the hidden units activated from the visible
    units in the data matrix passed in.
    """
    
    num_examples = data.shape[0]
    
    # Create a matrix, where each row is to be the hidden units (plus a bias unit)
    # sampled from a training example.
    hidden_states = np.ones((num_examples, self.num_hidden + 1))
    
    # Insert bias units of 1 into the first column of data.
    data = np.insert(data, 0, 1, axis = 1)

    # Calculate the activations of the hidden units.
    hidden_activations = np.dot(data, self.weights)
    # Calculate the probabilities of turning the hidden units on.
    hidden_probs = self._logistic(hidden_activations)
    # Turn the hidden units on with their specified probabilities.
    hidden_states[:,:] = hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
    # Always fix the bias unit to 1.
    # hidden_states[:,0] = 1
  
    # Ignore the bias units.
    hidden_states = hidden_states[:,1:]
    return hidden_states
    
  # TODO: Remove the code duplication between this method and `run_visible`?
  def run_hidden(self, data):
    """
    Assuming the RBM has been trained (so that weights for the network have been learned),
    run the network on a set of hidden units, to get a sample of the visible units.
    Parameters
    ----------
    data: A matrix where each row consists of the states of the hidden units.
    Returns
    -------
    visible_states: A matrix where each row consists of the visible units activated from the hidden
    units in the data matrix passed in.
    """

    num_examples = data.shape[0]

    # Create a matrix, where each row is to be the visible units (plus a bias unit)
    # sampled from a training example.
    visible_states = np.ones((num_examples, self.num_visible + 1))

    # Insert bias units of 1 into the first column of data.
    data = np.insert(data, 0, 1, axis = 1)

    # Calculate the activations of the visible units.
    visible_activations = np.dot(data, self.weights.T)
    # Calculate the probabilities of turning the visible units on.
    visible_probs = self._logistic(visible_activations)
    # Turn the visible units on with their specified probabilities.
    visible_states[:,:] = visible_probs > np.random.rand(num_examples, self.num_visible + 1)
    # Always fix the bias unit to 1.
    # visible_states[:,0] = 1

    # Ignore the bias units.
    visible_states = visible_states[:,1:]
    return visible_states
    
  def daydream(self, num_samples):
    """
    Randomly initialize the visible units once, and start running alternating Gibbs sampling steps
    (where each step consists of updating all the hidden units, and then updating all of the visible units),
    taking a sample of the visible units at each step.
    Note that we only initialize the network *once*, so these samples are correlated.
    Returns
    -------
    samples: A matrix, where each row is a sample of the visible units produced while the network was
    daydreaming.
    """

    # Create a matrix, where each row is to be a sample of of the visible units 
    # (with an extra bias unit), initialized to all ones.
    samples = np.ones((num_samples, self.num_visible + 1))

    # Take the first sample from a uniform distribution.
    samples[0,1:] = np.random.rand(self.num_visible)

    # Start the alternating Gibbs sampling.
    # Note that we keep the hidden units binary states, but leave the
    # visible units as real probabilities. See section 3 of Hinton's
    # "A Practical Guide to Training Restricted Boltzmann Machines"
    # for more on why.
    for i in range(1, num_samples):
      visible = samples[i-1,:]

      # Calculate the activations of the hidden units.
      hidden_activations = np.dot(visible, self.weights)      
      # Calculate the probabilities of turning the hidden units on.
      hidden_probs = self._logistic(hidden_activations)
      # Turn the hidden units on with their specified probabilities.
      hidden_states = hidden_probs > np.random.rand(self.num_hidden + 1)
      # Always fix the bias unit to 1.
      hidden_states[0] = 1

      # Recalculate the probabilities that the visible units are on.
      visible_activations = np.dot(hidden_states, self.weights.T)
      visible_probs = self._logistic(visible_activations)
      visible_states = visible_probs > np.random.rand(self.num_visible + 1)
      samples[i,:] = visible_states

    # Ignore the bias units (the first column), since they're always set to 1.
    return samples[:,1:]        
      
  def _logistic(self, x):
    return 1.0 / (1 + np.exp(-x))

if __name__ == '__main__':
  r = RBM(num_visible = 6, num_hidden = 2)
  training_data = np.array([[1,1,1,0,0,0],[1,0,1,0,0,0],[1,1,1,0,0,0],[0,0,1,1,1,0], [0,0,1,1,0,0],[0,0,1,1,1,0]])
  r.train(training_data, max_epochs = 5000)
  print(r.weights)
  user = np.array([[0,0,0,1,1,0]])
  print(r.run_visible(user))

Epoch 0: error is 9.10765448224169
Epoch 1: error is 8.808216708294319
Epoch 2: error is 8.575059407789759
Epoch 3: error is 8.424352543709514
Epoch 4: error is 8.227415256627681
Epoch 5: error is 8.15595178054042
Epoch 6: error is 7.8911700891305845
Epoch 7: error is 7.728681409354538
Epoch 8: error is 7.624404430785717
Epoch 9: error is 7.481886839064314
Epoch 10: error is 7.490550428964983
Epoch 11: error is 7.28949945050688
Epoch 12: error is 7.008243475383664
Epoch 13: error is 7.007623362342989
Epoch 14: error is 7.080403697248145
Epoch 15: error is 7.119503546688005
Epoch 16: error is 6.883996016327325
Epoch 17: error is 6.624666933981552
Epoch 18: error is 6.539536931400514
Epoch 19: error is 6.795790878553816
Epoch 20: error is 6.36088522735557
Epoch 21: error is 6.519668263068523
Epoch 22: error is 6.382109685603288
Epoch 23: error is 6.586067325230732
Epoch 24: error is 6.5242622755208695
Epoch 25: error is 6.411797601986635
Epoch 26: error is 6.206489633357897
Epoch 27: err

Epoch 858: error is 1.3531054528341775
Epoch 859: error is 1.352896238431938
Epoch 860: error is 1.3526953835205868
Epoch 861: error is 1.3525025535314774
Epoch 862: error is 1.352317426032974
Epoch 863: error is 1.3521396903845964
Epoch 864: error is 2.3771262707780436
Epoch 865: error is 1.3525905121562327
Epoch 866: error is 1.1196981990487598
Epoch 867: error is 1.3530704935055657
Epoch 868: error is 1.3528516160643163
Epoch 869: error is 1.3526414267636697
Epoch 870: error is 2.3728401402588823
Epoch 871: error is 1.353136486652326
Epoch 872: error is 1.3529096102276073
Epoch 873: error is 2.1282317284280188
Epoch 874: error is 1.35447096309309
Epoch 875: error is 1.3541813850282451
Epoch 876: error is 1.3539031470898235
Epoch 877: error is 1.3536358229666519
Epoch 878: error is 1.3533790000523869
Epoch 879: error is 1.353132279173036
Epoch 880: error is 1.7775398960711766
Epoch 881: error is 1.7581190692656272
Epoch 882: error is 1.3520205546042317
Epoch 883: error is 1.351805740

Epoch 1707: error is 1.3665427276769657
Epoch 1708: error is 0.8533175329330511
Epoch 1709: error is 0.8505505070744774
Epoch 1710: error is 1.368507583157236
Epoch 1711: error is 1.3674002428012837
Epoch 1712: error is 0.8510201993494693
Epoch 1713: error is 1.367834193416497
Epoch 1714: error is 1.3667482271484017
Epoch 1715: error is 0.8514515547720255
Epoch 1716: error is 1.367200311273835
Epoch 1717: error is 1.3661346537080172
Epoch 1718: error is 0.8518455779878085
Epoch 1719: error is 1.3666036909365349
Epoch 1720: error is 1.3655573259557268
Epoch 1721: error is 1.3645443091270468
Epoch 1722: error is 0.8537538763391785
Epoch 1723: error is 1.3650554226340188
Epoch 1724: error is 0.8524577152254356
Epoch 1725: error is 1.757293087956847
Epoch 1726: error is 0.8501980728105251
Epoch 1727: error is 1.3682552608910938
Epoch 1728: error is 1.3671528092148886
Epoch 1729: error is 1.3660848966389119
Epoch 1730: error is 1.3650508253448264
Epoch 1731: error is 1.3640498902590892
Epoc

Epoch 2583: error is 0.6996469029905827
Epoch 2584: error is 0.6994918240034544
Epoch 2585: error is 0.6993389992990635
Epoch 2586: error is 0.6991883832924372
Epoch 2587: error is 0.6990399315287725
Epoch 2588: error is 0.6988936006500617
Epoch 2589: error is 1.5135157786339652
Epoch 2590: error is 0.6993885921804631
Epoch 2591: error is 1.6223009327906002
Epoch 2592: error is 0.6983198876968141
Epoch 2593: error is 0.6981837054046617
Epoch 2594: error is 0.6980494019464571
Epoch 2595: error is 0.6979169408738776
Epoch 2596: error is 0.6977862866035813
Epoch 2597: error is 0.6976574043926977
Epoch 2598: error is 0.6975302603151164
Epoch 2599: error is 2.438690417924849
Epoch 2600: error is 0.6972810548023602
Epoch 2601: error is 0.697158929396054
Epoch 2602: error is 0.6970384141385373
Epoch 2603: error is 0.6969194788579887
Epoch 2604: error is 0.6968020940723938
Epoch 2605: error is 0.6966862309707008
Epoch 2606: error is 0.6965718613945691
Epoch 2607: error is 0.696458957820701
Epo

Epoch 3480: error is 0.6786031862438483
Epoch 3481: error is 0.6785851010641183
Epoch 3482: error is 1.5813332197976475
Epoch 3483: error is 0.6787345795793477
Epoch 3484: error is 0.6787149392750714
Epoch 3485: error is 0.678695427401435
Epoch 3486: error is 0.678676042574951
Epoch 3487: error is 0.678656783429812
Epoch 3488: error is 0.678637648617614
Epoch 3489: error is 0.678618636807092
Epoch 3490: error is 0.6785997466838557
Epoch 3491: error is 0.6785809769501329
Epoch 3492: error is 0.6785623263245183
Epoch 3493: error is 0.6785437935417261
Epoch 3494: error is 0.678525377352348
Epoch 3495: error is 0.6785070765226157
Epoch 3496: error is 1.6130771048105061
Epoch 3497: error is 0.6783024370421571
Epoch 3498: error is 0.6782861621780284
Epoch 3499: error is 0.6782699793320252
Epoch 3500: error is 0.6782538875681218
Epoch 3501: error is 0.678237885961502
Epoch 3502: error is 0.6782219735984064
Epoch 3503: error is 1.5843684618817657
Epoch 3504: error is 0.6783518594453354
Epoch 3

Epoch 4359: error is 0.6731876458120316
Epoch 4360: error is 0.6731824504844603
Epoch 4361: error is 0.6731772655762269
Epoch 4362: error is 0.6731720910299742
Epoch 4363: error is 0.6731669267916628
Epoch 4364: error is 0.6731617728103505
Epoch 4365: error is 0.6731566290379848
Epoch 4366: error is 1.6173932809072433
Epoch 4367: error is 0.6731057497843519
Epoch 4368: error is 1.6156541935082644
Epoch 4369: error is 0.673066957209168
Epoch 4370: error is 0.673062368495531
Epoch 4371: error is 0.6730577856155278
Epoch 4372: error is 0.6730532085821265
Epoch 4373: error is 0.6730486374085656
Epoch 4374: error is 0.6730440721082914
Epoch 4375: error is 0.6730395126949018
Epoch 4376: error is 0.6730349591820917
Epoch 4377: error is 0.6730304115836037
Epoch 4378: error is 0.6730258699131821
Epoch 4379: error is 0.673021334184529
Epoch 4380: error is 0.6730168044112647
Epoch 4381: error is 1.6135269255039453
Epoch 4382: error is 0.6730347670411959
Epoch 4383: error is 0.6730300621494393
Epo